*Copyright (c) Microsoft Corporation. All rights reserved.*  
*Licensed under the MIT License.*
# Named Entity Recognition Using BERT on Chinese
## Summary
This notebook demonstrates how to fine tune [pretrained BERT model](https://github.com/huggingface/pytorch-pretrained-BERT) for named entity recognition (NER) task on Chinese text. Utility functions and classes in the NLP Best Practices repo are used to facilitate data preprocessing, model training, model scoring and model evaluation.

[BERT (Bidirectional Transformers for Language Understanding)](https://arxiv.org/pdf/1810.04805.pdf) is a powerful pre-trained lanaguage model that can be used for multiple NLP tasks, including text classification, question answering, named entity recognition, etc. It's able to achieve state of the art performance with only a few epochs of fine tuning on task specific datasets.  
The figure below illustrates how BERT can be fine tuned for NER tasks. The input data is a list of tokens representing a sentence. In the training data, each token has an entity label. After fine tuning, the model predicts an entity label for each token in a given testing sentence. 

<img src="https://nlpbp.blob.core.windows.net/images/bert_architecture.png">

Named Entity Recognition on non-English text is not very differnt from that on English text. The only difference is the model used, which is configured by the `LANGUAGE` variable below. For non-English languages including Chinese, the *bert-base-multilingual-cased* model can be used by setting `LANGUAGE = Language.MULTILINGUAL`. For Chinese, the *bert-base-chinese* model can also be used by setting `LANGUAGE = Language.CHINESE`. On Chinese text, the performance of *bert-base-chinese* is usually better than *bert-base-multilingual-cased* because the *bert-base-chinese* model is pretrained on Chinese data only. On this particular dataset, the performances of the Chinese-only model and multilingual model are very similar

## Required packages
* pytorch
* pytorch-pretrained-bert
* pandas
* seqeval

In [1]:
import sys
import os
import random
from seqeval.metrics import classification_report

import torch

nlp_path = os.path.abspath('../../')
if nlp_path not in sys.path:
    sys.path.insert(0, nlp_path)

from utils_nlp.bert.token_classification import BERTTokenClassifier, postprocess_token_labels, create_label_map
from utils_nlp.bert.common import Language, Tokenizer
from utils_nlp.dataset.msra_ner import load_pandas_df, get_unique_labels

## Configurations

In [2]:
# path configurations
CACHE_DIR = "./temp"

# set random seeds
RANDOM_SEED = 100
torch.manual_seed(RANDOM_SEED)

# model configurations
LANGUAGE = Language.CHINESE
DO_LOWER_CASE = True
MAX_SEQ_LENGTH = 200

# training configurations
BATCH_SIZE = 16
NUM_TRAIN_EPOCHS = 1

# optimizer configuration
LEARNING_RATE = 3e-5

TEXT_COL = "sentence"
LABEL_COL = "labels"

## Preprocess Data

### Get training and testing data
The dataset used in this notebook is the MSRA NER dataset. The dataset consists of 45000 training sentences and 3442 testing sentences. 

The helper function `load_pandas_df` downloads the data files if they don't exist in `local_cache_path`. It returns the training or testing data frame based on `file_split`

The helper function `get_unique_labels` returns the unique entity labels in the dataset. There are 7 unique labels in the   dataset: 
* 'O': non-entity 
* 'B-LOC': beginning of location entity
* 'I-LOC': within location entity
* 'B-PER': beginning of person entity
* 'I-PER': within person entity
* 'B-ORG': beginning of organization entity
* 'I-ORG': within organization entity

The maximum number of words in a sentence is 2427. We set MAX_SEQ_LENGTH to 200 above to reduce the GPU memory needed to run this notebook. Less than 1% of testing data are longer than 200, so this should have negligible impact on the model performance evaluation.

In [3]:
train_df = load_pandas_df(local_cache_path=CACHE_DIR, file_split="train")
test_df = load_pandas_df(local_cache_path=CACHE_DIR, file_split="test")
label_list = get_unique_labels()
print("Number of sentences in training data: {}".format(train_df.shape[0]))
print("Number of sentences in testing data: {}".format(test_df.shape[0]))
print("Unique labels: {}".format(label_list))

Maximum sequence length in train data is: 746
Maximum sequence length in test data is: 2427
Number of sentences in training data: 45000
Number of sentences in testing data: 3442
Unique labels: ['O', 'B-LOC', 'B-ORG', 'B-PER', 'I-LOC', 'I-ORG', 'I-PER']


In [4]:
train_df.head()

,sentence,labels
0,"[当, 希, 望, 工, 程, 救, 助, 的, 百, 万, 儿, 童, 成, 长, 起, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,"[藏, 书, 本, 来, 就, 是, 所, 有, 传, 统, 收, 藏, 门, 类, 中, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,"[因, 有, 关, 日, 寇, 在, 京, 掠, 夺, 文, 物, 详, 情, ，, 藏, ...","[O, O, O, B-LOC, O, O, B-LOC, O, O, O, O, O, O..."
3,"[我, 们, 藏, 有, 一, 册, １, ９, ４, ５, 年, ６, 月, 油, 印, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[以, 家, 乡, 的, 历, 史, 文, 献, 、, 特, 定, 历, 史, 时, 期, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


### Tokenization and Preprocessing
The `tokenize_ner` method of the `Tokenizer` class converts raw string data to numerical features, involving the following steps:
1. WordPiece tokenization.
2. Convert tokens and labels to numerical values, i.e. token ids and label ids.
3. Sequence padding or truncation according to the `max_seq_length` configuration.

**Create a dictionary that maps labels to numerical values**

In [5]:
label_map = create_label_map(label_list)

**Tokenize input text**

In [6]:
tokenizer = Tokenizer(language=LANGUAGE, 
                      to_lower=DO_LOWER_CASE, 
                      cache_dir=CACHE_DIR)

**Create numerical features**  

In [7]:
train_token_ids, train_input_mask, train_trailing_token_mask, train_label_ids = \
    tokenizer.tokenize_ner(text=train_df[TEXT_COL],
                           label_map=label_map,
                           max_len=MAX_SEQ_LENGTH,
                           labels=train_df[LABEL_COL])
test_token_ids, test_input_mask, test_trailing_token_mask, test_label_ids = \
    tokenizer.tokenize_ner(text=test_df[TEXT_COL],
                           label_map=label_map,
                           max_len=MAX_SEQ_LENGTH,
                           labels=test_df[LABEL_COL])

`Tokenizer.tokenize_ner` outputs three or four lists of numerical features lists, each sublist contains features of an input sentence: 
1. token ids: list of numerical values each corresponds to a token.
2. attention mask: list of 1s and 0s, 1 for input tokens and 0 for padded tokens, so that padded tokens are not attended to. 
3. trailing word piece mask: boolean list, `True` for the first word piece of each original word, `False` for the trailing word pieces, e.g. ##ize. This mask is useful for removing predictions on trailing word pieces, so that each original word in the input text has a unique predicted label. 
4. label ids: list of numerical values each corresponds to an entity label, if `labels` is provided.

In [8]:
print(("Sample token ids:\n{}\n".format(train_token_ids[0])))
print("Sample attention mask:\n{}\n".format(train_input_mask[0]))
print("Sample trailing token mask:\n{}\n".format(train_trailing_token_mask[0]))
print("Sample label ids:\n{}\n".format(train_label_ids[0]))

Sample token ids:
[2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

## Create Token Classifier
The value of the `language` argument determines which BERT model is used:
* Language.ENGLISH: "bert-base-uncased"
* Language.ENGLISHCASED: "bert-base-cased"
* Language.ENGLISHLARGE: "bert-large-uncased"
* Language.ENGLISHLARGECASED: "bert-large-cased"
* Language.CHINESE: "bert-base-chinese"
* Language.MULTILINGUAL: "bert-base-multilingual-cased"

Here we use the base model pre-trained only on Chinese data.

In [9]:
token_classifier = BERTTokenClassifier(language=LANGUAGE,
                                       num_labels=len(label_map),
                                       cache_dir=CACHE_DIR)

## Train Model

In [10]:
token_classifier.fit(token_ids=train_token_ids, 
                     input_mask=train_input_mask, 
                     labels=train_label_ids,
                     num_epochs=NUM_TRAIN_EPOCHS, 
                     batch_size=BATCH_SIZE, 
                     learning_rate=LEARNING_RATE)

t_total value of -1 results in schedule not being applied
Iteration:   0%|          | 0/2813 [00:00<?, ?it/s]


Iteration:  56%|█████▌    | 1575/2813 [32:17<25:24,  1.23s/it]


Epoch: 100%|██████████| 1/1 [57:38<00:00, 3458.43s/it].23s/it]

Train loss: 0.07014742273803971


## Predict on Test Data
The `predict` method of the token classifier optionally returns the softmax probability of the predicted class, which is a NxM array, where N is size of the testing data and M is the number of tokens in the testing sequence

In [11]:
predictions = token_classifier.predict(token_ids=test_token_ids, 
                                       input_mask=test_input_mask, 
                                       labels=test_label_ids, 
                                       batch_size=BATCH_SIZE,
                                       probabilities=True)
probabilities = predictions.probabilities
pred_label_ids = predictions.classes
print("Sample predicted probabilities:\n{}\n".format(probabilities[5]))
print("Sample predicted labels:\n{}\n".format(pred_label_ids[5]))

Iteration:   0%|          | 0/216 [00:00<?, ?it/s]

Iteration: 100%|██████████| 216/216 [01:28<00:00,  2.45it/s]

Evaluation loss: 0.03504765126746389
Sample predicted probabilities:
[0.9922516  0.9720795  0.8725999  0.7776708  0.8586385  0.91872704
 0.9996006  0.9998628  0.99951696 0.99942696 0.9973278  0.9959351
 0.9978994  0.9984732  0.9971475  0.9968125  0.99599177 0.99230534
 0.98520017 0.9921949  0.9941393  0.9965797  0.9982722  0.9959889
 0.998398   0.9981147  0.9996132  0.99947053 0.9987999  0.9997645
 0.9997558  0.9998029  0.9998293  0.99985814 0.9997954  0.9994717
 0.9992329  0.9998041  0.99953985 0.99959916 0.9955279  0.98726255
 0.9762349  0.97292095 0.9446313  0.9501264  0.94189286 0.9638973
 0.9649065  0.98749185 0.99119383 0.99357057 0.99253017 0.9919087
 0.9881097  0.9850428  0.9849273  0.9764776  0.9668529  0.9686475
 0.96223867 0.9669822  0.98228574 0.9906934  0.99391055 0.99365366
 0.9967108  0.99211806 0.9812552  0.98241097 0.9620526  0.98186976
 0.98395514 0.98158413 0.9779302  0.95493126 0.9570392  0.9544368
 0.96813667 0.9672687  0.9663258  0.9765286  0.98573685 0.9870762
 0

## Evaluate Model
The `predict` method of the token classifier outputs label ids for all tokens, including the padded tokens. `postprocess_token_labels` is a helper function that removes the predictions on padded tokens. If a `label_map` is provided, it maps the numerical label ids back to original token labels which are usually string type. 

In [12]:
pred_tags_no_padding = postprocess_token_labels(pred_label_ids, 
                                                test_input_mask, 
                                                label_map)
true_tags_no_padding = postprocess_token_labels(test_label_ids, 
                                                test_input_mask, 
                                                label_map)
print(classification_report(true_tags_no_padding, pred_tags_no_padding, digits=2))

           precision    recall  f1-score   support

      LOC       0.92      0.93      0.92      2803
      PER       0.93      0.90      0.91      1382
      ORG       0.77      0.89      0.82      1296

micro avg       0.88      0.91      0.89      5481
macro avg       0.88      0.91      0.90      5481

